In [1]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar  9 09:38:03 2021

@author: oxide
"""

### Phonon mode simulations

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
#plt.style.use('/Users/oxide/Documents/research/orenstein/code/presentation_style_sheet.mplstyle')
palette = ['#58819b', '#d65f3d', '#92b26f', '#ac4050', '#d5bf66', '#bf7a8f', '#599ec4', '#3e8b67']*3

# anim parameters
a=1
kappa=1 
m=1
A=1/4*a
N_particles=15
interval=100 #ms 10 frames a second
frames = 200
plotscale=5

def omega(k):
    return 2*np.sqrt(kappa/m)*abs(np.sin(k*a/2))
def un(t, n, k):
    return np.real(A*np.exp(1j*omega(k)*t - 1j*k*n*a))
def xn(t, n, k):
    return a*n + un(t, n, k)


### Three different types of animation 

def animate_phonon(k):
    
    # set up bins
    k_vect = np.linspace(-2*np.pi/a, 2*np.pi/a, 1000)
    n_vect = np.arange(N_particles)
    u_vect = np.zeros(len(n_vect))
    x_vect = np.zeros(len(n_vect))
    xeq = a*n_vect

    scale=plotscale
    mosaic = [[f'{i}', f'{i}', 'A'] for i in [0,1]]
    fig, ax = plt.subplot_mosaic(mosaic,figsize=(1.2*scale,scale))

    ax['A'].plot(k_vect, omega(k_vect), color=palette[0], linewidth=5)
    ax['A'].axvline(np.pi/a)
    ax['A'].axvline(-np.pi/a)
    ax['A'].set(ylim=(-2,4))
    ax['A'].axhline(0,color='black')

    ax['A'].plot(k, omega(k), '*', color=palette[0], markersize=15)

    ax['A'].set_xticks([-np.pi/a, 0, np.pi/a], labels=[r'$-\frac{\pi}{a}$', '0', r'$\frac{\pi}{a}$'])
    ax['A'].set_yticks([])
    ax['A'].set(xlabel=r'wavevector $k$', ylabel=r'frequency $\omega(k)$')
    ax['A'].tick_params(axis='x', labelsize=18)

 
    # set up its axes etc
    ax[f'{0}'].set_xlabel('n (sites)')
    ax[f'{1}'].set_xlabel('n (sites)')
    ax[f'{0}'].set_ylabel(r'$\delta x_n$')
    ax[f'{0}'].set_yticks([])
    ax[f'{0}'].set_ylim(-2*A,2*A)
    ax[f'{0}'].set_xticks(xeq)
    ax[f'{1}'].set_xticks(xeq)
    ax[f'{1}'].set_yticks([])
    ax[f'{1}'].set_ylim(-0.001,0.001)
    for xc in xeq:
        ax[f'{1}'].axvline(x=xc, linestyle='--')
    ax[f'{0}'].axhline(0, linestyle='--')

    u_line, = ax['0'].plot([],[],'o', color=palette[1])
    x_line, = ax['1'].plot([],[],'o', color=palette[1])

    def animate(t): # animate at time t
        #print(t)
        t_actual = t*interval/1000
        # update bins
        for i in range(len(n_vect)):
            n = n_vect[i]
            u_vect[i] = un(t_actual,n,k)
            x_vect[i] = xn(t_actual,n,k)
        # plot bins
        u_line.set_data(n_vect, u_vect)
        x_line.set_data(x_vect, np.zeros(len(x_vect)))
        return u_line, x_line,
                
    fig.tight_layout()
    anim = FuncAnimation(fig, animate, frames=frames, interval=interval, blit=True)
    #writer = PillowWriter(fps=int(1000/interval))  
    plt.close(fig)
    return anim

def animate_phonons_dxn(Ks):

    Ks_color = palette[1:len(Ks)+1]
    
    # set up bins
    k_vect = np.linspace(-2*np.pi/a, 2*np.pi/a, 1000)
    n_vect = np.arange(N_particles)
    xeq = a*n_vect

    scale=plotscale
    mosaic = [[f'{i}', f'{i}', 'A'] for i in range(len(Ks))]
    fig, ax = plt.subplot_mosaic(mosaic,figsize=(1.2*scale,scale))

    ax['A'].plot(k_vect, omega(k_vect), color=palette[0], linewidth=5)
    ax['A'].axvline(np.pi/a)
    ax['A'].axvline(-np.pi/a)
    ax['A'].set(ylim=(-2,4))
    ax['A'].axhline(0,color='black')

    for ii, k in enumerate(Ks):
        ax['A'].plot(k, omega(k), '*', color=Ks_color[ii], markersize=15)

    ax['A'].set_xticks([-np.pi/a, 0, np.pi/a], labels=[r'$-\frac{\pi}{a}$', '0', r'$\frac{\pi}{a}$'])
    ax['A'].set_yticks([])
    ax['A'].set(xlabel=r'wavevector $k$', ylabel=r'frequency $\omega(k)$')
    ax['A'].tick_params(axis='x', labelsize=18)

    u_lines = []
    u_vects = []
    for ii, K in enumerate(Ks):
        # set up its axes etc
        ax[f'{ii}'].set_xlabel('n (sites)')
        ax[f'{ii}'].set_ylabel(r'$\delta x_n$')
        ax[f'{ii}'].set_yticks([])
        ax[f'{ii}'].set_ylim(-2*A,2*A)
        ax[f'{ii}'].set_xticks(xeq)
        for xc in xeq:
            ax[f'{ii}'].axvline(x=xc, linestyle='--')
        ax[f'{ii}'].axhline(0, linestyle='--')
            
        # set up artists
        u_line, = ax[f'{ii}'].plot([],[],'o', color=Ks_color[ii])
        u_vect = np.zeros(len(n_vect))
        u_lines.append(u_line)
        u_vects.append(u_vect)


    def animate(t): # animate at time t
        #print(t)
        t_actual = t*interval/1000
        # update bins
        for ii, k in enumerate(Ks):
            u_line = u_lines[ii]
            u_vect = u_vects[ii]
            for i in range(len(n_vect)):
                n = n_vect[i]
                u_vect[i] = un(t_actual,n,k)
            # plot bins
            u_line.set_data(n_vect, u_vect)
        return u_lines
                
    fig.tight_layout()
    anim = FuncAnimation(fig, animate, frames=frames, interval=interval, blit=True)
    #writer = PillowWriter(fps=int(1000/interval))  

    plt.close(fig)
    return anim

def animate_phonons_xn(Ks):

    Ks_color = palette[1:len(Ks)+1]
    
    # set up bins
    k_vect = np.linspace(-2*np.pi/a, 2*np.pi/a, 1000)
    n_vect = np.arange(N_particles)
    xeq = a*n_vect

    scale=plotscale
    mosaic = [[f'{i}', f'{i}', 'A'] for i in range(len(Ks))]
    fig, ax = plt.subplot_mosaic(mosaic,figsize=(1.2*scale,scale))

    ax['A'].plot(k_vect, omega(k_vect), color=palette[0], linewidth=5)
    ax['A'].axvline(np.pi/a)
    ax['A'].axvline(-np.pi/a)
    ax['A'].set(ylim=(-2,4))
    ax['A'].axhline(0,color='black')

    for ii, k in enumerate(Ks):
        ax['A'].plot(k, omega(k), '*', color=Ks_color[ii], markersize=15)

    ax['A'].set_xticks([-np.pi/a, 0, np.pi/a], labels=[r'$-\frac{\pi}{a}$', '0', r'$\frac{\pi}{a}$'])
    ax['A'].set_yticks([])
    ax['A'].set(xlabel=r'wavevector $k$', ylabel=r'frequency $\omega(k)$')
    ax['A'].tick_params(axis='x', labelsize=18)

    x_lines = []
    x_vects = []
    for ii, K in enumerate(Ks):
        # set up its axes etc
        ax[f'{ii}'].set_xlabel('n (sites)')
        ax[f'{ii}'].set_ylim(-0.001,0.001)
        ax[f'{ii}'].set_xticks(xeq)
        for xc in xeq:
            ax[f'{ii}'].axvline(x=xc, linestyle='--')
        ax[f'{ii}'].axhline(0, linestyle='--')
            
        # set up artists
        x_line, = ax[f'{ii}'].plot([],[],'o', color=Ks_color[ii])
        x_vect = np.zeros(len(n_vect))
        x_lines.append(x_line)
        x_vects.append(x_vect)


    def animate(t): # animate at time t
        #print(t)
        t_actual = t*interval/1000
        # update bins
        for ii, k in enumerate(Ks):
            x_line = x_lines[ii]
            x_vect = x_vects[ii]
            for i in range(len(n_vect)):
                n = n_vect[i]
                x_vect[i] = xn(t_actual,n,k)
            # plot bins
            x_line.set_data(x_vect, np.zeros(len(x_vect)))
        return x_lines
                
    fig.tight_layout()
    anim = FuncAnimation(fig, animate, frames=frames, interval=interval, blit=True)
    #writer = PillowWriter(fps=int(1000/interval))  

    plt.close(fig)
    return anim

# animations

In [ ]:
anim = animate_phonon(np.pi/4)
HTML(anim.to_jshtml())

In [ ]:
anim = animate_phonon(np.pi/16)
HTML(anim.to_jshtml())

In [ ]:
anim = animate_phonons_dxn([-np.pi/4, np.pi/4])
HTML(anim.to_jshtml())

In [ ]:
anim = animate_phonons_dxn([-np.pi/(a*4), -np.pi/(a*4) + 2*np.pi/(a)])
HTML(anim.to_jshtml())